In [2]:
import pandas as pd
from datetime import datetime
from datetime import date
import json
import math
import xlrd
import numpy as np

hubs_dict = {'ARICA': 'IQUIQUE',
 'IQUIQUE': 'IQUIQUE',
 'ANTOFAGASTA': 'ANTOFAGASTA',
 'COPIAPO': 'COPIAPO',
 'COQUIMBO': 'COQUIMBO',
 'OVALLE': 'COQUIMBO',
 'ILLAPEL': 'CURAUMA',
 'LLAY LLAY': 'CURAUMA',
 'CURAUMA': 'CURAUMA',
 'CERVECERA': 'SANTIAGO SUR',
 'MODELO': 'SANTIAGO SUR',
 'SANTIAGO SUR': 'SANTIAGO SUR',
 'RANCAGUA': 'SANTIAGO SUR',
 'TALCA': 'TALCA',
 'CHILLAN': 'TALCAHUANO',
 'TALCAHUANO': 'TALCAHUANO',
 'LOS ANGELES': 'TALCAHUANO',
 'TEMUCO': 'TEMUCO',
 'VALDIVIA': 'VALDIVIA',
 'OSORNO': 'VALDIVIA',
 'PUERTO MONTT': 'PUERTO MONTT',
 'CASTRO': 'PUERTO MONTT',
 'COYHAIQUE': 'COYHAIQUE',
 'CALAMA': 'ANTOFAGASTA'}

sheets_dict1 = pd.read_excel('../data/venta SKU por CD 2018.xlsx', sheet_name=None)
"""sheets_dict2 = pd.read_excel('../data/venta SKU por CD 2019.xlsx', sheet_name=None)
sheets_dict3 = pd.read_excel('../data/venta SKU por CD 2020.xlsx', sheet_name=None)"""

master = pd.DataFrame()
for name in sheets_dict1:
    
    df = sheets_dict1[name]
    master = pd.concat([master, df])
    
print(master)

       DESCR_CENDIST     FECHA  ID_SKU_VENTA  \
0         TALCAHUANO  12/01/18        870353   
1           COQUIMBO  27/01/18        875148   
2            CURAUMA  15/01/18          1325   
3           COQUIMBO  18/01/18        870426   
4          LLAY LLAY  06/01/18        870421   
...              ...       ...           ...   
670272   ANTOFAGASTA  07/12/18        870178   
670273        TEMUCO  01/12/18        450024   
670274         ARICA  12/12/18        450208   
670275       CURAUMA  06/12/18        450576   
670276       CURAUMA  15/12/18        446139   

                                     DESCR_PROD  Venta en cajas  
0         870353 - WATTS PIÑA LIGHT PET 1500X06            80.0  
1        875148 - PEPSI-BILZ-PAP 3PF-PET3000-TR           155.0  
2                     1325 - PEPSI 6PF-PET 2000           153.0  
3       870426 - VIVO BERRIES MENTA 100 SOBX7GR             5.0  
4       870421 - VIVO FRUTILLA 100 SOBRES X 7GR            15.0  
...                        

In [3]:
master = master.drop(columns='DESCR_PROD')

In [61]:
master['DESCR_CENDIST'].unique()

array(['TALCAHUANO', 'COQUIMBO', 'CURAUMA', 'LLAY LLAY', 'MODELO',
       'RANCAGUA', 'SANTIAGO SUR', 'OVALLE', 'ILLAPEL', 'CALAMA',
       'LOS ANGELES', 'LA VARA', 'TALCA', 'PUERTO MONTT', 'ANTOFAGASTA',
       'COYHAIQUE', 'CERVECERA', 'COPIAPO', 'CHILLAN', 'TEMUCO',
       'IQUIQUE', 'VALDIVIA', 'OSORNO', 'ARICA', 'LO RUIZ', 'CASTRO',
       'CL8000', 'OLIVO', 'VTA ONLINE', 'PATAGONA'], dtype=object)

In [7]:
datos_agrupados = master.groupby(by=['DESCR_CENDIST', 'ID_SKU_VENTA', 'FECHA']).sum().reset_index()

In [8]:
datos_agrupados

,DESCR_CENDIST,ID_SKU_VENTA,FECHA,Venta en cajas
0,ANTOFAGASTA,515,01/02/18,3.0
1,ANTOFAGASTA,515,01/03/18,7.0
2,ANTOFAGASTA,515,01/06/18,3.0
3,ANTOFAGASTA,515,01/08/18,4.0
4,ANTOFAGASTA,515,01/09/18,1.0
...,...,...,...,...
4862276,VTA ONLINE,875219,30/08/18,2.0
4862277,VTA ONLINE,875219,30/10/18,3.0
4862278,VTA ONLINE,875219,31/07/18,2.0
4862279,VTA ONLINE,875219,31/08/18,4.0


In [13]:
def date_from_string(string):
    lista = string.split('/')
    if lista[2] == '18':
        return 2018, int(lista[1]), int(lista[0])
    elif lista[2] == '19':
        return 2019, int(lista[1]), int(lista[0])
    elif lista[2] == '20':
        return 2020, int(lista[1]), int(lista[0])

In [14]:
datos_agrupados['FECHA'] = datos_agrupados.apply(lambda x: date(*date_from_string(x.FECHA)), axis=1)


In [15]:
datos_agrupados

,DESCR_CENDIST,ID_SKU_VENTA,FECHA,Venta en cajas
0,ANTOFAGASTA,515,2018-02-01,3.0
1,ANTOFAGASTA,515,2018-03-01,7.0
2,ANTOFAGASTA,515,2018-06-01,3.0
3,ANTOFAGASTA,515,2018-08-01,4.0
4,ANTOFAGASTA,515,2018-09-01,1.0
...,...,...,...,...
4862276,VTA ONLINE,875219,2018-08-30,2.0
4862277,VTA ONLINE,875219,2018-10-30,3.0
4862278,VTA ONLINE,875219,2018-07-31,2.0
4862279,VTA ONLINE,875219,2018-08-31,4.0


In [16]:
datos_agrupados = datos_agrupados.astype({'FECHA': 'datetime64[ns]'})

In [19]:
with open('../data/skus_filter.json', 'r') as file:
    skus_filter = set(json.load(file))

In [20]:
cds_validos = set(hubs_dict.keys())

In [21]:
datos_agrupados = datos_agrupados[(~datos_agrupados['ID_SKU_VENTA'].isin(skus_filter)) & (datos_agrupados['DESCR_CENDIST'].isin(cds_validos))]

In [22]:
datos_agrupados

,DESCR_CENDIST,ID_SKU_VENTA,FECHA,Venta en cajas
0,ANTOFAGASTA,515,2018-02-01,3.0
1,ANTOFAGASTA,515,2018-03-01,7.0
2,ANTOFAGASTA,515,2018-06-01,3.0
3,ANTOFAGASTA,515,2018-08-01,4.0
4,ANTOFAGASTA,515,2018-09-01,1.0
...,...,...,...,...
4810489,VALDIVIA,875220,2018-10-31,5.0
4810490,VALDIVIA,875225,2018-01-09,2.0
4810491,VALDIVIA,875225,2018-10-09,3.0
4810492,VALDIVIA,875225,2018-10-19,2.0


In [24]:
datos_agrupados['verano'] = datos_agrupados.apply(lambda x: x.FECHA.month in set((1, 2, 12)), axis=1)

In [26]:
datos_verano = datos_agrupados[datos_agrupados.verano == True]

In [28]:
datos_verano = datos_verano.drop(columns='verano')

In [29]:
datos_verano

,DESCR_CENDIST,ID_SKU_VENTA,FECHA,Venta en cajas
0,ANTOFAGASTA,515,2018-02-01,3.0
6,ANTOFAGASTA,515,2018-12-01,1.0
7,ANTOFAGASTA,515,2018-01-02,4.0
8,ANTOFAGASTA,515,2018-02-02,8.0
15,ANTOFAGASTA,515,2018-01-03,5.0
...,...,...,...,...
4810471,VALDIVIA,875220,2018-01-29,30.0
4810478,VALDIVIA,875220,2018-12-29,1.0
4810479,VALDIVIA,875220,2018-01-30,3.0
4810486,VALDIVIA,875220,2018-01-31,4.0


In [32]:
datos_verano = datos_verano.sort_values(by=['DESCR_CENDIST', 'ID_SKU_VENTA', 'FECHA'])

In [34]:
datos_verano_copy = datos_verano.copy()

In [35]:
cdList, skuList, fechasList, zeros = [], [], [], []
for cd in datos_verano['DESCR_CENDIST'].unique():
    for sku in datos_verano['ID_SKU_VENTA'].unique():
        for fecha in datos_verano['FECHA'].unique():
            cdList.append(cd)
            skuList.append(sku)
            fechasList.append(fecha)
            zeros.append(0)

In [36]:
aux_dict = {'DESCR_CENDIST': cdList, 'ID_SKU_VENTA': skuList, 'FECHA': fechasList, 'Venta en cajas': zeros}

In [37]:
aux_dataframe = pd.DataFrame.from_dict(aux_dict)

In [38]:
datos_verano = pd.concat([datos_verano, aux_dataframe])

In [40]:
datos_verano_fill_zeros = datos_verano.groupby(by=['DESCR_CENDIST', 'ID_SKU_VENTA', 'FECHA']).sum().reset_index()

In [41]:
datos_verano_fill_zeros

,DESCR_CENDIST,ID_SKU_VENTA,FECHA,Venta en cajas
0,ANTOFAGASTA,515,2018-01-02,4.0
1,ANTOFAGASTA,515,2018-01-03,5.0
2,ANTOFAGASTA,515,2018-01-04,11.0
3,ANTOFAGASTA,515,2018-01-05,16.0
4,ANTOFAGASTA,515,2018-01-06,0.0
...,...,...,...,...
2420995,VALDIVIA,875556,2018-12-26,0.0
2420996,VALDIVIA,875556,2018-12-27,0.0
2420997,VALDIVIA,875556,2018-12-28,0.0
2420998,VALDIVIA,875556,2018-12-29,0.0


In [43]:
conversion = pd.read_csv('../data/conversiones_unidades.csv', encoding = "ISO-8859-1")[['ID_SKU_VENTA', 'CAJAS_POR_PALLET']]
sku_pallets = {}
for index, row in conversion.iterrows():
    sku_pallets[row['ID_SKU_VENTA']] = row['CAJAS_POR_PALLET']
    
datos_verano_fill_zeros['Venta en cajas'] = datos_verano_fill_zeros.apply(lambda x: x['Venta en cajas'] / sku_pallets[x['ID_SKU_VENTA']], axis=1)

In [44]:
datos_verano_fill_zeros

,DESCR_CENDIST,ID_SKU_VENTA,FECHA,Venta en cajas
0,ANTOFAGASTA,515,2018-01-02,0.100
1,ANTOFAGASTA,515,2018-01-03,0.125
2,ANTOFAGASTA,515,2018-01-04,0.275
3,ANTOFAGASTA,515,2018-01-05,0.400
4,ANTOFAGASTA,515,2018-01-06,0.000
...,...,...,...,...
2420995,VALDIVIA,875556,2018-12-26,0.000
2420996,VALDIVIA,875556,2018-12-27,0.000
2420997,VALDIVIA,875556,2018-12-28,0.000
2420998,VALDIVIA,875556,2018-12-29,0.000


In [45]:
datos_verano_fill_zeros = datos_verano_fill_zeros.rename(columns={'Venta en cajas': 'Venta en pallets'})

In [63]:
datos_verano_fill_zeros.to_csv('../data/test.csv')

In [64]:
test = pd.read_csv('../data/test.csv')

In [65]:
test

,Unnamed: 0,DESCR_CENDIST,ID_SKU_VENTA,FECHA,Venta en pallets
0,0,ANTOFAGASTA,515,2018-01-02,0.100
1,1,ANTOFAGASTA,515,2018-01-03,0.125
2,2,ANTOFAGASTA,515,2018-01-04,0.275
3,3,ANTOFAGASTA,515,2018-01-05,0.400
4,4,ANTOFAGASTA,515,2018-01-06,0.000
...,...,...,...,...,...
2420995,2420995,VALDIVIA,875556,2018-12-26,0.000
2420996,2420996,VALDIVIA,875556,2018-12-27,0.000
2420997,2420997,VALDIVIA,875556,2018-12-28,0.000
2420998,2420998,VALDIVIA,875556,2018-12-29,0.000


In [49]:
def stats(datos):

    mean_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).mean().reset_index().rename(columns={'Venta en pallets': 'MEDIA'})
    stdev_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).std().reset_index().rename(columns={'Venta en pallets': 'STD'})

    mean_no_group['MEDIA'] = mean_no_group.apply(lambda x: x.MEDIA * 7, axis=1)
    stdev_no_group['STD'] = stdev_no_group.apply(lambda x: x.STD * 7, axis=1)

    data_completa = mean_no_group.merge(stdev_no_group, on=['ID_SKU_VENTA', 'DESCR_CENDIST'])

    data_completa['COEF_VAR'] = data_completa.apply(lambda x: x['STD'] / x['MEDIA'] if x['MEDIA'] != 0 else 0, axis=1)

    data_completa['SS'] = data_completa.apply(lambda x: 1.65 * x['STD'], axis=1)

    data_completa['STOCK_TEORICO'] = data_completa.apply(lambda x: x['MEDIA'] + x['SS'], axis=1)

    data_completa['PORCENTAJE_SS_DEL_TOTAL'] = data_completa.apply(lambda x: x['SS'] / x['STOCK_TEORICO'] if x['MEDIA'] != 0 else 0, axis=1)

    porcentaje_ss_del_total = data_completa['SS'].sum() / data_completa['STOCK_TEORICO'].sum()
    
    return data_completa, porcentaje_ss_del_total

In [50]:
datos_verano_stats_no_agrupado, porcentaje_ss_total_no_agrupado = stats(datos_verano_fill_zeros)

In [51]:
datos_verano_stats_no_agrupado

,ID_SKU_VENTA,DESCR_CENDIST,MEDIA,STD,COEF_VAR,SS,STOCK_TEORICO,PORCENTAJE_SS_DEL_TOTAL
0,515,ANTOFAGASTA,1.134583,0.805951,0.710350,1.329819,2.464402,0.539611
1,515,ARICA,0.149333,0.317373,2.125269,0.523666,0.673000,0.778108
2,515,CALAMA,0.781667,0.712501,0.911515,1.175627,1.957293,0.600639
3,515,CASTRO,0.548333,0.604395,1.102239,0.997251,1.545584,0.645226
4,515,CERVECERA,26.355000,8.870768,0.336588,14.636767,40.991767,0.357066
...,...,...,...,...,...,...,...,...
32275,875556,SANTIAGO SUR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
32276,875556,TALCA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
32277,875556,TALCAHUANO,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
32278,875556,TEMUCO,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [52]:
porcentaje_ss_total_no_agrupado

0.5440002835666343

In [54]:
dict_sku_cd = {}
#for index, row in master.iterrows():
for index, row in datos_verano_fill_zeros.iterrows():
    
    #if row['CANTIDAD HUB 3R'] != 0:
    #if row['COEF_VAR'] >= 0:
    dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 1
        
    """else:
        dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 0"""

KeyboardInterrupt: 

In [56]:
datos_verano_fill_zeros_agrupados = datos_verano_fill_zeros.copy()
datos_verano_fill_zeros_agrupados['DESCR_CENDIST'] = datos_verano_fill_zeros.apply(lambda x: hubs_dict[x['DESCR_CENDIST']], axis=1)
datos_verano_fill_zeros_agrupados = datos_verano_fill_zeros_agrupados.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'FECHA']).sum().reset_index()

KeyError: 'PERIODO'

In [57]:
datos_verano_fill_zeros_agrupados = datos_verano_fill_zeros_agrupados.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'FECHA']).sum().reset_index()

In [58]:
datos_verano_stats_agrupado, porcentaje_ss_total_agrupado = stats(datos_verano_fill_zeros_agrupados)

In [59]:
datos_verano_stats_agrupado

,ID_SKU_VENTA,DESCR_CENDIST,MEDIA,STD,COEF_VAR,SS,STOCK_TEORICO,PORCENTAJE_SS_DEL_TOTAL
0,515,ANTOFAGASTA,1.916250,1.062416,0.554425,1.752986,3.669236,0.477752
1,515,COPIAPO,0.725667,0.744518,1.025978,1.228455,1.954121,0.628648
2,515,COQUIMBO,3.210667,2.041136,0.635736,3.367874,6.578541,0.511949
3,515,COYHAIQUE,0.186667,0.241278,1.292561,0.398109,0.584775,0.680789
4,515,CURAUMA,0.424667,0.486689,1.146050,0.803037,1.227704,0.654097
...,...,...,...,...,...,...,...,...
16135,875556,SANTIAGO SUR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16136,875556,TALCA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16137,875556,TALCAHUANO,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16138,875556,TEMUCO,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [60]:
porcentaje_ss_total_agrupado

0.5064099916188347